In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm import tqdm

In [2]:
# URL of Earth.org that need to be scraped 
INIT_URL = 'https://earth.org/climate-change/'

# Prevent URL from detecting scraper as a bot when requesting 
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.148 Safari/537.36'}

In [5]:
# Earth.org contains many articles that are divided into many sub_pages
sub_URL_track = 0 # to track each sub_pages

# Create dataframe to hold scraped data
df = pd.DataFrame()

# Loop through each sub_page
while(True):
    
    # Tracking sub_page
    sub_URL_track += 1
    
    print("===================== Processing sub_page number {} ======================".format(sub_URL_track))
    
    # Each sub_page differs from others by extra element in its URL: "sf_paged"
    s_page = requests.get('{}?sf_paged={}'.format(INIT_URL, sub_URL_track), headers = HEADERS) # remember to pass the defined HEADERS
    soup = BeautifulSoup(s_page.content, "html.parser")
    
    # Each sub_page contains many articles with their unique URL
    try:
        # Find all tags containing articles within this sub_page
        df['Page {}'.format(sub_URL_track)] = soup.find_all('article')
    except:
        break

# Main part: scrape article using its corresponding URL 
def scrape_article(URL):
    
    # Scraping request 
    a_page = requests.get(URL, headers = HEADERS)
    soup = BeautifulSoup(a_page.content, "html.parser")
    
    # Keep necessary information only
    title, image, content = map(lambda x: soup.find('section', class_=x), 
                                     ['post-hero','post_image','post-content'])

    return {'title'  : '{}'.format(title),
            'content': '{}{}'.format(image, content),
            'URL'    : URL}        

# tqdm helps observe progress of applying function on dataframe 
tqdm.pandas()

# Map each cell in dataframe to its corresponding article URL then perform scraping for each URL accordingly
df = df.progress_applymap(lambda x: scrape_article(x.find('a')['href']))

===================== Processing sub_page number 1 ======================
===================== Processing sub_page number 2 ======================
===================== Processing sub_page number 3 ======================
===================== Processing sub_page number 4 ======================
===================== Processing sub_page number 5 ======================
===================== Processing sub_page number 6 ======================
===================== Processing sub_page number 7 ======================
===================== Processing sub_page number 8 ======================
===================== Processing sub_page number 9 ======================
===================== Processing sub_page number 10 ======================
===================== Processing sub_page number 11 ======================
===================== Processing sub_page number 12 ======================
===================== Processing sub_page number 13 ======================
===================== Processing s

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [01:37<00:00,  3.07it/s]
